In [32]:
import random
import os
import sys
import time
import numpy as np
import math
import zlib
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay

In [33]:
from CurveCGen import *
from CurveEA import *

In [34]:
import subprocess
BLENDER_PATH="C:\\Program Files\\Blender Foundation\\Blender 3.3\\blender.exe"

In [35]:
OUTPUT_PATH="E:\\Research\\Statue Generator\\Generations"

In [36]:
def shannon_entropy(file_path):
    with open(file_path, "rb") as f:
        bytes = f.read()
    prob = [float(bytes.count(b)) / len(bytes) for b in set(bytes)]
    entropy = -sum([p * math.log(p) / math.log(2.0) for p in prob])
    return round(entropy,6)

In [37]:
def golden_ratio(vertices, edges):
    def euclidean_distance(v1, v2):
        return ((v2[0] - v1[0])**2 + (v2[1] - v1[1])**2 + (v2[2] - v1[2])**2)**0.5
        
    golden_ratio = 1.61803398874989484820
    ratio_sum = 0
    ratio_count = 0
    
    for edge in edges:
        v1 = vertices[edge[0]]
        v2 = vertices[edge[1]]
        length = euclidean_distance(v1, v2)
        ratio = length / golden_ratio
        if ratio >= 1:
            ratio = 1 / ratio
        ratio_sum += ratio
        ratio_count += 1
    
    return ratio_sum / ratio_count


In [38]:
def perimeter(vertices, edges):
    perimeter = 0
    for edge in edges:
        vertex1, vertex2 = edge
        x1, y1, z1 = vertices[vertex1]
        x2, y2, z2 = vertices[vertex2]
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
        perimeter += distance
    return perimeter

In [39]:
from collections import defaultdict

def tetrahedron_volume(a, b, c, d):
    return np.abs(np.dot(a-d, np.cross(b-d, c-d))) / 6

def volume_3d(vertices, edges):
    adjacency_list = defaultdict(list)
    for edge in edges:
        adjacency_list[edge[0]].append(edge[1])
        adjacency_list[edge[1]].append(edge[0])
    faces = []
    for vertex, neighbors in adjacency_list.items():
        for i in range(len(neighbors)):
            for j in range(i+1, len(neighbors)):
                if neighbors[j] in adjacency_list[neighbors[i]]:
                    faces.append([vertex, neighbors[i], neighbors[j]])
    total_volume = 0
    for face in faces:
        a, b, c = [vertices[vertex] for vertex in face]
        d = np.average(vertices, axis=0)
        total_volume += tetrahedron_volume(a, b, c, d)
    return total_volume

In [40]:
def volume(vertices):
    hull = ConvexHull(vertices)
    return hull.volume

In [41]:
def volume_3d_object(vertices):
    tri = Delaunay(vertices)
    tetrahedrons = vertices[tri.simplices.astype(int)]
    volume = 0
    for tetrahedron in tetrahedrons:
        v1, v2, v3, v4 = tetrahedron
        volume += np.abs(np.dot(v1 - v4, np.cross(v2 - v4, v3 - v4))) / 6
    return volume

In [42]:
def compression_ratio(filepath):
    with open(filepath, mode="rb") as fin, open(filepath[:-6]+"_compressed", mode="wb") as fout:
            data = fin.read()
            compressed_data = zlib.compress(data, zlib.Z_BEST_COMPRESSION)
            orig=sys.getsizeof(data)
            comp=sys.getsizeof(compressed_data)
            ratio=comp/orig
    
            return ratio

In [43]:
def gen_blender(ARR,GEN_NUM):
    
    os.mkdir(OUTPUT_PATH+"\\GEN_"+str(GEN_NUM))
    
    for i,C in enumerate(ARR):
        
        #[r,g,b,a,bd,me,ro,ss,[VERTICES,EDGES],[new_L,new_B,new_H]]

        VERTS=C[8][0]
        EDGES=C[8][1]
        MATERIAL=str([C[0],C[1],C[2],C[3]])
        BD=str(C[4])
        ME=str(C[5])
        RO=str(C[6])
        SS=str(C[7])
        FILE_NUM=str(i)
        
        with open(f'{OUTPUT_PATH}\\GEN_{GEN_NUM}\\VERTS_G{GEN_NUM}_S{i}.txt', 'w') as f:
            f.writelines(str(VERTS))

        subprocess.run(["blender", "--background", "--python","generate.py",FILE_NUM,f'VERTS_G{GEN_NUM}_S{i}.txt',MATERIAL,BD,ME,RO,SS,str(OUTPUT_PATH+"\\GEN_"+str(GEN_NUM))],capture_output=True,shell=True)
        #time.sleep(60)
        
        #print(f'Statue {i} Saved!')

## Basic Parameters for generation

In [44]:
L,B,H=10,12,17
POPULATION_SIZE=10

In [45]:
PARAMS=[]

## Initial Population

In [46]:
%%time

INITIAL_POPULATION=[]

for _ in range(POPULATION_SIZE):
    CH=generate_chromosome(L,B,H)
    INITIAL_POPULATION.append(CH)
    get_stats(CH)
    print('-'*125)

PARAMS.append(INITIAL_POPULATION)

(R, G, B, A) = (0.523167, 0.636074, 0.12743, 0.656943)
Number of vertices = 8
Number of edges = 7
Bevel Depth = 1.45
Metallic = 0.345062
Roughness = 0.254635
Subsurface Level = 2
New Bounding Box - (L,B,H) = (8.939225,7.192993,2.620968)
-----------------------------------------------------------------------------------------------------------------------------
(R, G, B, A) = (0.597512, 0.328823, 0.088814, 0.324541)
Number of vertices = 4
Number of edges = 3
Bevel Depth = 1.37
Metallic = 0.920916
Roughness = 0.190982
Subsurface Level = 2
New Bounding Box - (L,B,H) = (3.09112,8.686615,1.315393)
-----------------------------------------------------------------------------------------------------------------------------
(R, G, B, A) = (0.186947, 0.405591, 0.482187, 0.401323)
Number of vertices = 8
Number of edges = 7
Bevel Depth = 1.61
Metallic = 0.734542
Roughness = 0.260389
Subsurface Level = 3
New Bounding Box - (L,B,H) = (5.812175,1.166738,12.908073)
-----------------------------------

In [47]:
%%time

os.mkdir(OUTPUT_PATH+"\\GEN_0")

for i,C in enumerate(PARAMS[0]):

    VERTS=C[8][0]
    EDGES=C[8][1]
    MATERIAL=str([C[0],C[1],C[2],C[3]])
    BD=str(C[4])
    ME=str(C[5])
    RO=str(C[6])
    SS=str(C[7])
    FILE_NUM=str(i)
    
    with open(f'{OUTPUT_PATH}\\GEN_0\\VERTS_G{0}_S{i}.txt', 'w') as f:
        f.writelines(str(VERTS))

    #subprocess.run(["blender", "--background", "--python","convexGeneratePopulation.py",FILE_NUM,f'VERTS_G{GEN_NUM}_S{i}.txt',f'FACES_G{GEN_NUM}_S{i}.txt',f'MATERIALS_G{GEN_NUM}_S{i}.txt',str(RO),str(OUTPUT_PATH+"\\GEN_"+str(GEN_NUM))],capture_output=True,shell=True)
    subprocess.run(["blender", "--background", "--python","generate.py",FILE_NUM,f'VERTS_G0_S{i}.txt',MATERIAL,BD,ME,RO,SS,str(OUTPUT_PATH+"\\GEN_0")],capture_output=True,shell=True)
    #subprocess.run(["blender", "--background", "--python","convexGeneratePopulation.py",FILE_NUM,f'VERTS_G{0}_S{i}.txt',f'FACES_G{0}_S{i}.txt',f'MATERIALS_G{0}_S{i}.txt',str(RO),str(OUTPUT_PATH+"\\GEN_0")],capture_output=True,shell=True)
    #print(subprocess.run(["blender", "--background", "--python","generatePopulation.py",FILE_NUM,COLOUR,f'VERTS_G{0}_S{i}.txt',f'FACES_G{0}_S{i}.txt',str(RO),str(OUTPUT_PATH+"\\GEN_0")],capture_output=True,shell=True))
    #time.sleep(60)
    
    print(f'Statue {i} Saved!')
    

Statue 0 Saved!
Statue 1 Saved!
Statue 2 Saved!
Statue 3 Saved!
Statue 4 Saved!
Statue 5 Saved!
Statue 6 Saved!
Statue 7 Saved!
Statue 8 Saved!
Statue 9 Saved!
Wall time: 42.4 s


## Crossover and Mutation for more generations

In [55]:
GENERATIONS=5

In [49]:
%%time

ALL_FITNESS=[]
BEST_FITNESS=[]
GEN_NUM=1

for _ in range(GENERATIONS):
    
    PATH_FOR_PREV_GEN=OUTPUT_PATH+"\\GEN_"+str(GEN_NUM-1)
    FITNESS=[]
    
    for i in range(POPULATION_SIZE):
        
        VERTS=PARAMS[GEN_NUM-1][i][8][0]
        EDGES=PARAMS[GEN_NUM-1][i][8][1]
        
#         entropy=shannon_entropy(PATH_FOR_PREV_GEN+f"\\STATUE_{i}.blend")
#         entropy=entropy/8 
        
#         cr=compression_ratio(PATH_FOR_PREV_GEN+f"\\STATUE_{i}.blend")
        
#         gr=golden_ratio(VERTS,EDGES)
        
        v=volume_3d(VERTS,EDGES)
        p=perimeter(VERTS,EDGES)
        print(f"Volume: {v}")
        print(f"Perimeter: {p}")
    
        #fitness=volume+perimeter
        fitness=p/v
        FITNESS.append(round(fitness,6))
    
    ALL_FITNESS.append(FITNESS)
    
    BEST_FITNESS.append(FITNESS.index(max(FITNESS)))
    print(f'Statue-{FITNESS.index(max(FITNESS))} has the best fitness value in {GEN_NUM-1} generation.')
    
    MODELS=[]
    
    for _ in range(int(POPULATION_SIZE/2)):
    
        #selecting 2 random parents from previous generation
        idx1,idx2=random.randint(0,POPULATION_SIZE-1),random.randint(0,POPULATION_SIZE-1)
        p1=idx1 if FITNESS[idx1]>FITNESS[idx2] else idx2
        
        idx3,idx4=random.randint(0,POPULATION_SIZE-1),random.randint(0,POPULATION_SIZE-1)
        p2=idx3 if FITNESS[idx3]>FITNESS[idx4] else idx4
        
        crossover_prob=round(random.random(),6)
        mutation_prob=round(random.random(),6)
        
        if mutation_prob<0.8:
            #mutate the children after crossover
            m1=mutate(PARAMS[GEN_NUM-1][p1],L,B,H)
            m2=mutate(PARAMS[GEN_NUM-1][p2],L,B,H)
        else:
            m1,m2=PARAMS[GEN_NUM-1][p1],PARAMS[GEN_NUM-1][p2]
        
        if crossover_prob<0.2:
            #perform crossover for the selected parents
            c1,c2=crossover(m1,m2)
        else:
            c1,c2=m1,m2
        
        
        
        MODELS.append(c1)
        MODELS.append(c2)
        
    #print(len(MODELS))
    #appending the mutated childs to PARAMS array
    PARAMS.append(MODELS)
    
    #generate blender file
    gen_blender(MODELS,GEN_NUM)
    
    print(f"Generation {GEN_NUM} generated successfully!")
    print("-"*120)
    
    GEN_NUM+=1

Statue-0 has the best fitness value in 0 generation.


<timed exec>:26: RuntimeWarning: divide by zero encountered in double_scalars


Generation 1 generated successfully!
------------------------------------------------------------------------------------------------------------------------
Statue-0 has the best fitness value in 1 generation.
Generation 2 generated successfully!
------------------------------------------------------------------------------------------------------------------------
Statue-0 has the best fitness value in 2 generation.
Generation 3 generated successfully!
------------------------------------------------------------------------------------------------------------------------
Statue-0 has the best fitness value in 3 generation.


KeyboardInterrupt: 

In [50]:
%%time
PATH_FOR_PREV_GEN=OUTPUT_PATH+"\\GEN_"+str(GENERATIONS)
FINAL_FITNESS=[]

for i in range(POPULATION_SIZE):

    entropy=shannon_entropy(PATH_FOR_PREV_GEN+f"\\STATUE_{i}.blend")
    entropy=entropy/8

#     VERTS=PARAMS[GEN_NUM-1][i][8][0]
#     EDGES=PARAMS[GEN_NUM-1][i][8][1]

#     angle=calculate_angles(VERTS,EDGES)

    FINAL_FITNESS.append(round(entropy,6))

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Research\\Statue Generator\\Generations\\GEN_300\\STATUE_0.blend'

In [51]:
FINAL_FITNESS

[]

In [52]:
ALL_FITNESS.append(FINAL_FITNESS)

In [53]:
FINAL_FITNESS.index(max(FINAL_FITNESS))

ValueError: max() arg is an empty sequence

## Plot fitness

In [ ]:
import matplotlib.pyplot as plt
def plot_fitness(fitness):
    
    best_values=[]
    for g in fitness:
        best_values.append(max(g))
    
    plt.figure(figsize=(20,10))
    plt.plot(best_values)
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.show()

In [ ]:
plot_fitness(ALL_FITNESS)

In [ ]:
ALL_FITNESS.pop()

## Saving the parameters

In [ ]:
with open(f'E:\Research\Statue gen output\\NewEA_1.txt', 'w') as f:
        f.writelines(str(PARAMS))